# Base Model Seq2Seq


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import string
!pip install unidecode
import unidecode
import numpy as np
import itertools
from collections import Counter
from tqdm import tqdm



# Building blocks

First, we will define the encode and decoder

<img src="" width="25%">

In [2]:
NULL_INDEX = 0

encoder_dim = 1024
predictor_dim = 1024
joiner_dim = 1024

In [3]:
from torch import nn
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [33]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell
        

In [34]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.95):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos>
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
INPUT_DIM = 32
OUTPUT_DIM = 32
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 1024
N_LAYERS = 2
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2
START_INDEX = 0
BLANK_INDEX = 27
PAD_INDEX = 30
END_INDEX = 31

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

In [36]:
def greedy_search(self, x, T):
  y_batch = []
  B = len(x)
  print(B, len(T))  

  encoder_out = self.encoder.forward(x)
  U_max = 200
  for b in range(B):
    t = 0; u = 0; y = [self.predictor.start_symbol]; predictor_state = self.predictor.initial_state.unsqueeze(0)
    while t < T[b] and u < U_max:
      predictor_input = torch.tensor([ y[-1] ]).to(x.device)
      g_u, predictor_state = self.predictor.forward_one_step(predictor_input, predictor_state)
      f_t = encoder_out[b, t]
      h_t_u = self.joiner.forward(f_t, g_u)
      argmax = h_t_u.max(-1)[1].item()
      if argmax == NULL_INDEX:
        t += 1
      else: # argmax == a label
        u += 1
        y.append(argmax)
    y_batch.append(y[1:]) # remove start symbol
  return y_batch

#Transducer.greedy_search = greedy_search

# Some utilities

Here we will add a bit of boilerplate code for training and loading data.

In [37]:
import random
char_labels = ['<start>', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's',
              't', 'u', 'v', 'w', 'x', 'y', 'z', '<blank>', ' ', "'", '<pad>','<end>']
char_dict = {'<start>':0, 'a':1, 'b':2, 'c':3, 'd':4, 'e':5, 'f':6, 'g':7, 'h':8, 'i':9, 'j':10, 'k':11, 'l':12, 'm':13, 
             'n':14, 'o':15, 'p':16, 'q':17, 'r':18, 's':19, 't':20, 'u':21, 'v':22, 'w':23, 'x':24, 'y':25, 'z':26, 
             '<blank>':27, ' ':28, "'":29, '<pad>':30,'<end>':31}
char_inv_dict = {0:'', 1:'a', 2:'b', 3:'c', 4:'d', 5:'e', 6:'f', 7:'g', 8:'h', 9:'i', 10:'j', 11:'k', 12:'l',
                 13:'m', 14:'n', 15:'o', 16:'p', 17:'q', 18:'r', 19:'s', 20:'t', 21:'u', 22:'v', 23:'w', 24:'x', 25:'y',
                 26:'z', 27:'', 28:' ', 29:"'", 30:'', 31:''}

class TextDataset(torch.utils.data.Dataset):
  def __init__(self, lines, batch_size):
    lines = list(filter(("\n").__ne__, lines))

    self.lines = lines # list of strings
    collate = Collate()
    self.loader = torch.utils.data.DataLoader(self, batch_size=batch_size, num_workers=0, shuffle=True, collate_fn=collate)

  def __len__(self):
    return len(self.lines)

  def __getitem__(self, idx):
    line = self.lines[idx].replace("\n", "")
    line =line.lower()
    line = "".join(c for c in line if c in char_labels)
    line = unidecode.unidecode(line) # remove special characters
    x = "".join(c*random.randint(1,2) for c in line) 
    x = ' '.join(x.split())
    y = line
    return (x,y)

def encode_string(s):

  return [char_dict[c] for c in s]

def decode_labels(l):
  return "".join([char_inv_dict[c] for c in l])

class Collate:
  def __call__(self, batch):
    """
    batch: list of tuples (input string, output string)
    Returns a minibatch of strings, encoded as labels and padded to have the same length.
    """
    x = []; y = []
    batch_size = len(batch)
    for index in range(batch_size):
      x_,y_ = batch[index]
      x.append([0]+encode_string(x_))
      y.append([0]+encode_string(y_))

    # pad all sequences to have same length
    T = [len(x_) for x_ in x]
    U = [len(y_) for y_ in y]
 
    T_max = max(T)
    U_max = max(U)
    for index in range(batch_size):
      x[index] += [PAD_INDEX] * (T_max - len(x[index])) +[31]
      x[index] = torch.tensor(x[index])
      y[index] += [PAD_INDEX] * (U_max - len(y[index]))+[31]
      y[index] = torch.tensor(y[index])

    # stack into single tensor
    x = torch.stack(x)
    y = torch.stack(y)
    #T = torch.tensor(T)
    #U = torch.tensor(U)

    return (x,y) #,T,U)

filename="../LibriSpeech/train-clean-100/transcripts_460.tsv"
with open(filename, "r") as f:
   t_lines = f.readlines()
t_lines = t_lines[1:]
t_lines = [line.split('\t')[2] for line in t_lines ]
lines = []

for line in t_lines:
    temp = line.split(' ')
    count = len(temp)//10
    resid = len(temp)%10
    
    k = 0
    while(count > 0):    
        short_line = ' '.join(temp[k:k+10]) 
        lines.append(short_line)
        count = count - 1
        k = k + 10
    if(resid > 0):
        lines.append(' '.join(temp[k:k+resid]))   

end = round(0.9 * len(lines))
train_lines = lines[:end]
test_lines = lines[end:]
train_set = TextDataset(train_lines, batch_size=32)
test_set = TextDataset(test_lines, batch_size=8)
train_set.__getitem__(0)

('weell pprinnccee soo ggeenoaa annd lucccca aarree nnow jjuusst fammillyy esstaatteess of tthe',
 'well prince so genoa and lucca are now just family estates of the')

In [38]:
print(len(train_lines), len(test_lines))

55814 6201


In [39]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(32, 256)
    (rnn): LSTM(256, 1024, num_layers=2, dropout=0.2)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(32, 256)
    (rnn): LSTM(256, 1024, num_layers=2, dropout=0.2)
    (fc_out): Linear(in_features=1024, out_features=32, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [40]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss(ignore_index = PAD_INDEX)

lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.95)

The model has 27,344,928 trainable parameters


In [41]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src, trg = batch
        trg = trg.transpose(1,0)
        src = src.transpose(1,0)
        optimizer.zero_grad()
        src = src.to(device)
        trg = trg.to(device)
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].reshape(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)



def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src, trg = batch
            trg = trg.transpose(1,0)
            src = src.transpose(1,0)
            src = src.to(device)
            trg = trg.to(device)
            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].reshape(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


In [50]:
N_EPOCHS = 100
CLIP = 1

best_valid_loss = float('inf')
train_loss = []
valid_loss = []
for epoch in range(N_EPOCHS):
    
    tr_loss = train(model, train_set.loader, optimizer, criterion, CLIP)
    train_loss.append(tr_loss)
    v_loss = evaluate(model, test_set.loader, criterion)
    valid_loss.append(v_loss)
    lr_scheduler.step()
    print(tr_loss, v_loss)
    if(best_valid_loss > v_loss):
        best_valid_loss = v_loss
        save_file = './base_model.pt'
        torch.save(model.state_dict(), save_file)
print(train_loss, valid_loss)

1.3374481074228273 5.297865208807021
1.3136368889902812 5.292362336429108
1.2797559591375387 5.313594293000171
1.2409358961962174 5.285190923191677
1.2147206622891769 5.277388626169936
1.198586932778863 5.287266259624208
1.1775167363739147 5.319936996679811
1.1601882145250129 5.404023858988396
1.1440086207537792 5.487968570346773
1.1331019845012213 5.3515930539722385
1.1173553513875296 5.2927597660884675
1.1030308015339803 5.413021916540984
1.0928088345463758 5.3956130055995
1.0781456292600322 5.399575396861614
1.0654458912622777 5.3620887352298725
1.055238149734411 5.356019328316424
1.0445925931752316 5.428011049734097
1.0353394867839194 5.422454022915564
1.026464333024785 5.4612542647067635


KeyboardInterrupt: 

In [52]:
outputs.shape
a = outputs.argmax(-1)
a1 = a.transpose(1,0)

In [53]:
t = trg.transpose(1,0)
a = a1.cpu()
b = t.cpu()
b = b.detach().numpy()
a = a.detach().numpy()
for i in range(a.shape[0]):
   print(decode_labels(a[i,:]))
   print(decode_labels(b[i,:]))
   print("------------------") 

place in the morning of the french army when they were all right and a
platon karataev of whom he had heard from pierre his love for natasha
------------------
who had been at the same time he was an air of a man who was already an
whole face as she had so often seen it in the stillness of the night
------------------
have you heard of the same i shall be in a man who was a single and
has lived through
------------------
french soldiers were silent and silently still his head and heard
freedom
------------------
a commander of the rostovs were standing the french and the soldiers
a happiness he had long forgotten and of which he had not even been
------------------
what a story of the emperor alexander in moscow and what is it asked th
what pierre had told them princess mary did not express her opinion of
------------------
speaking of his face and the french were already been able to sing and
special enclosure where there are oatsthat this very ram swelling
------------------
authorit

In [13]:
for idx, batch in enumerate(train_set.loader):
      src,trg = batch
      if(idx == 0):
        break

In [15]:
for i in range(2):
   print(src[i])
         

tensor([ 0,  1,  4,  4, 22,  1, 14, 20, 20,  1,  1,  7,  7,  5,  5, 15, 21, 21,
        19, 28, 19, 20, 20, 25, 12, 12,  5,  5, 28, 12,  9, 11, 11,  5,  5, 28,
         1, 28, 23,  5, 12, 12, 12, 12,  7,  1,  1, 18, 14, 14,  9,  9, 19, 19,
         8,  5,  5,  4,  4, 28, 10, 10, 15,  9, 14, 20, 20, 28, 15,  6, 28, 18,
        15,  1,  1, 19, 20, 20, 28,  2,  5,  5,  5,  6, 28, 15, 14, 28,  1,  1,
        28,  8, 15, 20, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 31])
tensor([ 0, 20, 20,  8,  5,  5, 28, 18, 15, 15, 15, 13, 13, 28, 19, 20,  5,  5,
        16, 16, 16,  9,  9, 14, 14,  7, 28,  2,  2, 18,  9,  9, 19, 11, 12, 25,
        28, 20, 20, 15, 28, 20,  8,  5, 28,  6, 18, 18, 15, 15, 14, 20, 28, 15,
         6, 28, 20, 20,  8,  8,  5, 28,  3,  3, 18, 15, 15, 23,  4, 28, 15, 15,
         6, 28,  7,  5,  5, 14, 14, 20, 20, 18, 25, 25, 30, 30, 30, 30, 30, 30,
        30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 31])


In [ ]:
b.extend([2])
b

In [ ]:
a = [i for i in range(6)]

In [ ]:
0.95**(100)